# Random Forest Regressor Model

In [50]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import mean_squared_error

## Loading the data

In [51]:
df1 = pd.read_csv("../../data/TestsetCleanFinal.csv", sep=",")

In [52]:
df1.head()

,ActualLandingTime,Airline,AircraftType,RunwayNumber,StandNumber,Date,AWNDmean,PRCPmean,TAVGmean,WDF5mean,...,IceRoad,WetDay,ShipmentWeightCat,NbPlanesLast10Mn,Hour,WeekDay,ParkingArea,StandRank,TaxiTime,LogTaxiTime
0,2019-01-17 03:18:00,CAR1,A321/2,9.0,128,2019-01-17 00:00:00,2.61,0.043,42.0,223.333333,...,1.0,1.0,M,0.0,3,3,16337.8,1,5.0,1.791759
1,2019-01-17 03:34:00,CAR1,CS100,9.0,37,2019-01-17 00:00:00,2.61,0.043,42.0,223.333333,...,1.0,1.0,L,0.0,3,3,13110.0,0,8.0,2.197225
2,2019-01-17 03:36:00,CAR1,A321/2,9.0,9,2019-01-17 00:00:00,2.61,0.043,42.0,223.333333,...,1.0,1.0,M,1.0,3,3,16337.8,1,7.0,2.079442
3,2019-01-17 03:43:00,CAR1,CS100,0.0,28,2019-01-17 00:00:00,2.61,0.043,42.0,223.333333,...,1.0,1.0,L,2.0,3,3,13110.0,0,5.0,1.791759
4,2019-01-17 03:44:00,CAR1,MD88,9.0,32,2019-01-17 00:00:00,2.61,0.043,42.0,223.333333,...,1.0,1.0,L,2.0,3,3,15739.7,1,5.0,1.791759


We drop the AircraftType. We already put this information in the parking area column.

In [53]:
df1.drop(['AircraftType'], axis=1, inplace=True)

We drop useless columns

In [54]:
df1.drop(['ActualLandingTime', 'Date', 'StandNumber'], axis=1, inplace=True)

In [55]:
df1.shape

(8904, 22)

## One-Hot Encoding

As the Random Forest Regressor of scikit learn does not support categorical variables, we use One-Hot encoding.

In [56]:
df1.dtypes

Airline               object
RunwayNumber         float64
AWNDmean             float64
PRCPmean             float64
TAVGmean             float64
WDF5mean             float64
WSF2max              float64
WSF5max              float64
WT01max              float64
WT02max              float64
WT03max              float64
WT08max              float64
IceRoad              float64
WetDay               float64
ShipmentWeightCat     object
NbPlanesLast10Mn     float64
Hour                   int64
WeekDay                int64
ParkingArea          float64
StandRank              int64
TaxiTime             float64
LogTaxiTime          float64
dtype: object

In [57]:
df1['RunwayNumber'] = pd.Categorical(df1['RunwayNumber'])
df1['Airline'] = pd.Categorical(df1['Airline'])
df1['ShipmentWeightCat'] = pd.Categorical(df1['ShipmentWeightCat'])

In [58]:
dfDummiesRunwayNumber = pd.get_dummies(df1['RunwayNumber'], prefix = 'RunwayNumber')
dfDummiesAirline = pd.get_dummies(df1['Airline'], prefix = 'Airline')
dfDummiesShipmentWeightCat = pd.get_dummies(df1['ShipmentWeightCat'], prefix = 'ShipmentWeightCat')

In [59]:
df2 = pd.concat([df1, dfDummiesRunwayNumber, dfDummiesAirline, dfDummiesShipmentWeightCat], axis=1)

In [60]:
df2.shape

(8904, 38)

One column is missing because an Airline Company disapeared in the testing set (the CAR2 one). We need to add a column.

In [61]:
df2['Airline_CAR2']=0

In [62]:
df2.head()

,Airline,RunwayNumber,AWNDmean,PRCPmean,TAVGmean,WDF5mean,WSF2max,WSF5max,WT01max,WT02max,...,RunwayNumber_7.0,RunwayNumber_8.0,RunwayNumber_9.0,Airline_CAR1,Airline_CAR3,Airline_CAR4,ShipmentWeightCat_L,ShipmentWeightCat_M,ShipmentWeightCat_S,Airline_CAR2
0,CAR1,9.0,2.61,0.043,42.0,223.333333,8.9,13.0,1.0,0.0,...,0,0,1,1,0,0,0,1,0,0
1,CAR1,9.0,2.61,0.043,42.0,223.333333,8.9,13.0,1.0,0.0,...,0,0,1,1,0,0,1,0,0,0
2,CAR1,9.0,2.61,0.043,42.0,223.333333,8.9,13.0,1.0,0.0,...,0,0,1,1,0,0,0,1,0,0
3,CAR1,0.0,2.61,0.043,42.0,223.333333,8.9,13.0,1.0,0.0,...,0,0,0,1,0,0,1,0,0,0
4,CAR1,9.0,2.61,0.043,42.0,223.333333,8.9,13.0,1.0,0.0,...,0,0,1,1,0,0,1,0,0,0


## Creat input matrix and target columns

In [63]:
X = df2.drop(['TaxiTime', 'LogTaxiTime', 'RunwayNumber', 'Airline', 'ShipmentWeightCat'], axis=1)
y = df2['LogTaxiTime']

In [64]:
print(X.shape)
print(y.shape)

(8904, 34)
(8904,)


## Random Forest Regressor

Load the model

In [65]:
loaded_rf_model = pickle.load(open('finalized_random_forest_model.sav', 'rb'))

In [66]:
y_pred = np.exp(loaded_rf_model.predict(X))-1

In [67]:
y_pred[:10]

array([6.84490067, 6.92462227, 6.84490067, 6.31306098, 6.77118829,
       6.82330394, 6.70174249, 6.60736269, 6.74936527, 6.12106459])

#### Compute RMSE score

In [68]:
np.sqrt(mean_squared_error(np.exp(y)-1, y_pred))

3.9045349180659215

#### Compute accuracy +/- 3 and 5 minutes

In [69]:
accuracy_dataframe = pd.DataFrame(data={'y_pred':y_pred, 'y':np.exp(y)-1})

In [70]:
accuracy_dataframe['3MinutesRange'] = abs(accuracy_dataframe['y_pred'] - accuracy_dataframe['y']) <= 3
accuracy_dataframe['5MinutesRange'] = abs(accuracy_dataframe['y_pred'] - accuracy_dataframe['y']) <= 5

In [71]:
print(accuracy_dataframe['3MinutesRange'].value_counts(normalize=True) * 100)

print(accuracy_dataframe['5MinutesRange'].value_counts(normalize=True) * 100)

True     81.221923
False    18.778077
Name: 3MinutesRange, dtype: float64
True     92.980683
False     7.019317
Name: 5MinutesRange, dtype: float64
